In [133]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math, time
import datetime
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM

# fixar random seed para se puder reproduzir os resultados
seed = 9
np.random.seed(seed)

In [212]:
def read_csv_dataset(file_name):
    df = pd.read_csv(file_name)
    print('Formato do dataset: ',df.shape)
    print('Feature Engineering...')
    date_split = df['Month'].str.split('-').str
    df['Year'], df['Month'] = date_split
    m = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12, }
    df['Month'] = df['Month'].map(m)
    print('Formato do dataset: ',df.shape)
    df.drop(df.columns[[3,4,5,6]], axis=1, inplace=True) #vou só ficar com as colunas 0,1,2,6
    df.drop(df.tail(2).index,inplace=True) #eliminar as duas últimas linhas com lixo
    df.dropna() #just to be sure
    df["Year"] = df["Year"].astype(dtype=np.float64) #converter coluna do ano para floats
    print(df.values)
    return df

In [213]:
#função load_data do lstm.py configurada para aceitar qualquer número de parametros
#o último atributo é que fica como label (resultado)
#stock é um dataframe do pandas (uma especie de dicionario + matriz)
#seq_len é o tamanho da janela a ser utilizada na serie temporal
def load_data(df_dados, janela):
    qt_atributos = len(df_dados.columns)
    mat_dados = df_dados.as_matrix() #converter dataframe para matriz (lista com lista de cada registo)
    tam_sequencia = janela + 1
    res = []
    for i in range(len(mat_dados) - tam_sequencia): #numero de registos - tamanho da sequencia
        res.append(mat_dados[i: i + tam_sequencia])
    res = np.array(res) #dá como resultado um np com uma lista de matrizes(janela deslizante ao longo da serie)
    qt_casos_treino = int(round((2/3) * res.shape[0])) #2/3 passam a ser casos de treino
    train = res[:qt_casos_treino, :]
    x_train = train[:, :-1] #menos um registo pois o ultimo registo é o registo a seguir à janela
    y_train = train[:, -1][:,-1] #para ir buscar o último atributo para a lista dos labels
    x_test = res[qt_casos_treino:, :-1]
    y_test = res[qt_casos_treino:, -1][:,-1]
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], qt_atributos))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], qt_atributos))
    return [x_train, y_train, x_test, y_test]

In [214]:
# Etapa 2 - Definir a topologia da rede (arquitectura do modelo) e compilar '''
def build_model2(janela):
    model = Sequential()
    model.add(LSTM(128, input_shape=(janela, 4), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(64, input_shape=(janela, 4), return_sequences=False))
    #model.add(Dropout(0.2))
    model.add(Dense(16, activation="relu", kernel_initializer="uniform"))
    model.add(Dense(1, activation="linear", kernel_initializer="uniform"))
    model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
    return model

In [215]:
#imprime um grafico com os valores de teste e com as correspondentes tabela de previsões
def print_series_prediction(y_test,predic):
    diff=[]
    racio=[]
    for i in range(len(y_test)): #para imprimir tabela de previsoes
        racio.append( (y_test[i]/predic[i])-1)
        diff.append( abs(y_test[i]- predic[i]))
        print('valor: %f ---> Previsão: %f Diff: %f Racio: %f' % (y_test[i],predic[i], diff[i],racio[i]))
    plt.plot(y_test,color='blue', label='y_test')
    plt.plot(predic,color='red', label='prediction') #este deu uma linha em branco
    plt.plot(diff,color='green', label='diff')
    plt.plot(racio,color='yellow', label='racio')
    plt.legend(loc='upper left')
    plt.show()

In [216]:
#util para visualizar a topologia da rede num ficheiro em pdf ou png
def print_model(model,fich):
    from keras.utils import plot_model
    plot_model(model, to_file=fich, show_shapes=True, show_layer_names=True)

In [217]:
'''
MSE- (Mean square error), RMSE- (root mean square error) –
o significado de RMSE depende do range da label. para o mesmo range menor é melhor.
'''
def LSTM():
    df = read_csv_dataset("advertising-and-sales-data-36-co.csv")
    janela = 22 #tamanho da Janela deslizante
    X_train, y_train, X_test, y_test = load_data(df[::-1], janela)# o df[::-1] é o df por ordem inversa
    print("df", df.shape)
    print("Formato do X_train", X_train.shape)
    print("Formato do y_train", y_train.shape)
    print("Formato do X_test", X_test.shape)
    print("Formato do y_test", y_test.shape)
    model = build_model2(janela)
    model.fit(X_train, y_train, batch_size=512, epochs=500, validation_split=0.1, verbose=1)
    print_model(model,"lstm_model.png")
    trainScore = model.evaluate(X_train, y_train, verbose=0)
    print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore[0], math.sqrt(trainScore[0])))
    testScore = model.evaluate(X_test, y_test, verbose=0)
    print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore[0], math.sqrt(testScore[0])))
    print(model.metrics_names)
    p = model.predict(X_test)
    predic = np.squeeze(np.asarray(p)) #para transformar uma matriz de uma coluna e n linhas em um np array de n elementos
    print_series_prediction(y_test,predic)

In [218]:
if __name__ == '__main__':
    #visualize_GOOGL()
    LSTM()

Formato do dataset:  (38, 7)
Feature Engineering...
Formato do dataset:  (38, 8)
[[ 1.  12.  15.   1. ]
 [ 2.  20.5 16.   1. ]
 [ 3.  21.  18.   1. ]
 [ 4.  15.5 27.   1. ]
 [ 5.  15.3 21.   1. ]
 [ 6.  23.5 49.   1. ]
 [ 7.  24.5 21.   1. ]
 [ 8.  21.3 22.   1. ]
 [ 9.  23.5 28.   1. ]
 [10.  28.  36.   1. ]
 [11.  24.  40.   1. ]
 [12.  15.5  3.   1. ]
 [ 1.  17.3 21.   2. ]
 [ 2.  25.3 29.   2. ]
 [ 3.  25.  62.   2. ]
 [ 4.  36.5 65.   2. ]
 [ 5.  36.5 46.   2. ]
 [ 6.  29.6 44.   2. ]
 [ 7.  30.5 33.   2. ]
 [ 8.  28.  62.   2. ]
 [ 9.  26.  22.   2. ]
 [10.  21.5 12.   2. ]
 [11.  19.7 24.   2. ]
 [12.  19.   3.   2. ]
 [ 1.  16.   5.   3. ]
 [ 2.  20.7 14.   3. ]
 [ 3.  26.5 36.   3. ]
 [ 4.  30.6 40.   3. ]
 [ 5.  32.3 49.   3. ]
 [ 6.  29.5  7.   3. ]
 [ 7.  28.3 52.   3. ]
 [ 8.  31.3 65.   3. ]
 [ 9.  32.2 17.   3. ]
 [10.  26.4  5.   3. ]
 [11.  23.4 17.   3. ]
 [12.  16.4  1.   3. ]]
df (36, 4)
Formato do X_train (9, 22, 4)
Formato do y_train (9,)
Formato do X_test (4, 22,

C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


TypeError: LSTM() got an unexpected keyword argument 'input_shape'